In [ ]:
class NearestNeighborClassifierManual:
    def __init__(self):
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        #To Do
        for test_instance in X_test:
      # Calculate distances to all training points
          distances = np.linalg.norm(self.X_train - test_instance, axis=1)

      # Find k nearest neighbors
          k_nearest_indices = np.argsort(distances)[:k]

      # Get nearest neighbor labels
          nearest_neighbor_labels = self.y_train[k_nearest_indices]

      # Majority vote for prediction
          prediction = np.argmax(np.bincount(nearest_neighbor_labels))

          predictions.append(prediction)
    return np.array(predictions)


# Import GaussianNaiveBayesClassifierManual
class GaussianNaiveBayesClassifierManual:
    def __init__(self):
        self.class_priors = None
        self.class_means = None
        self.class_variances = None

    def fit(self, X_train, y_train):
        self.class_priors = {}
        self.class_means = {}
        self.class_variances = {}
        classes = np.unique(y_train)
        for c in classes:
      # Get data points belonging to class c
          X_c = X_train[y_train == c]

      # Calculate class prior
          self.class_priors[c] = len(X_c) / len(y_train)

      # Calculate class mean
          self.class_means[c] = np.mean(X_c, axis=0)

      # Calculate class variance (add epsilon for smoothing)
          self.class_variances[c] = np.var(X_c, axis=0) + 1e-10


    def predict(self, X_test):
        predictions = []
        #To Do
        for x in X_test:
          posteriors = {}
          for c in self.class_priors:
        # Calculate probability density for each feature
            pdfs = np.exp(-0.5 * np.sum(((x - self.class_means[c])**2) / (self.class_variances[c] + 1e-10), axis=0))

        # Class prior * product of feature probabilities
            posteriors[c] = self.class_priors[c] * np.prod(pdfs)

      # Predict class with highest posterior probability
          prediction = max(posteriors, key=posteriors.get)
          predictions.append(prediction)
        return np.array(predictions)


class SupportVectorMachineClassifierManual:
    def __init__(self, learning_rate=0.001, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        #To Do
            # Loop for training epochs
        for _ in range(self.epochs):
          for i in range(n_samples):
        # Get current sample and label
            x_i = X_train[i]
            y_i = y_train[i]

        # Predicted value
            predicted = np.dot(self.weights, x_i) + self.bias

        # Update weights only if prediction violates margin
            if (y_i == 1 and predicted < 1) or (y_i == -1 and predicted > -1):
          # Update rule with hinge loss
              update = self.learning_rate * (y_i - predicted) * x_i
              self.weights += update
              self.bias += self.learning_rate * (y_i - predicted)


    def predict(self, X_test):
        #To Do
          predictions = []
    for x in X_test:
      predicted = np.dot(self.weights, x) + self.bias
      # Apply sign function for binary classification
      prediction = np.sign(predicted)
      predictions.append(prediction)
    return np.array(predictions)


class ConfusionMatrix:
    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
        self.n_classes = len(np.unique(y_true))
        self.matrix = self._compute_confusion_matrix()

    def _compute_confusion_matrix(self):
        matrix = np.zeros((self.n_classes, self.n_classes), dtype=int)

       for y_t, y_p in zip(self.y_true, self.y_pred):
         matrix[y_t, y_p] += 1  # Increment count for actual class (row) and predicted class (column)
       return matrix
    def plot(self):
        plt.figure(figsize=(8, 6))
        sns.heatmap(self.matrix, annot=True, cmap='Blues', fmt='d', xticklabels=np.arange(self.n_classes), yticklabels=np.arange(self.n_classes))
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()



class EvaluationMetrics:
    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
        self.confusion_matrix = ConfusionMatrix(y_true, y_pred)
        self.metrics = self._compute_metrics()

    def _compute_metrics(self):
        tp = np.diag(self.confusion_matrix.matrix)
        tn = cm.sum(axis=0) - tp  # True negatives (correct non-predictions) - sum across rows (predicted class) excluding diagonal
        fp = cm.sum(axis=1) - tp  # False positives (incorrect predictions) - sum across columns (true class) excluding diagonal
        fn = cm.sum(axis=0) - tn  # False negatives (missed positives) - sum across rows excluding diagonal


            # Derive other metrics from basic counts
        sensitivity = tp / (tp + fn)  # True Positive Rate (TPR), Recall
        specificity = tn / (tn + fp)  # True Negative Rate (TNR)
        fpr = fp / (fp + tn)  # False Positive Rate (FPR)
        fnr = fn / (tp + fn)  # False Negative Rate (FNR)
        precision = tp / (tp + fp)  # Positive Predictive Value (PPV)
        recall = tp / (tp + fn)
        f1_score = 2 * (precision * recall) / (precision + recall)

        return {
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'FPR': fpr,
            'FNR': fnr,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1_score
        }



#Write a main to test all the above functions
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate sample classification data
X, y = make_classification(n_samples=100, n_features=10, n_classes=2, random_state=42)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a KNN classifier object
knn_classifier = NearestNeighborClassifierManual()
knn_classifier.fit(X_train, y_train)
y_pred_knn = knn_classifier.predict(X_test)

# Create a GNB classifier object
gnb_classifier = GaussianNaiveBayesClassifierManual()
gnb_classifier.fit(X_train, y_train)
y_pred_gnb = gnb_classifier.predict(X_test)

# Print evaluation metrics for KNN
print("KNN Classifier Evaluation:")
evaluation_metrics_knn = EvaluationMetrics(y_test, y_pred_knn)
print(evaluation_metrics_knn.metrics)

# Print evaluation metrics for GNB
print("GNB Classifier Evaluation:")
evaluation_metrics_gnb = EvaluationMetrics(y_test, y_pred_gnb)
print(evaluation_metrics_gnb.metrics)

# (Optional) Confusion matrix visualization (requires matplotlib and seaborn)
# You can uncomment these lines to plot the confusion matrices
 import matplotlib.pyplot as plt
 import seaborn as sns

 knn_classifier.confusion_matrix.plot()
 gnb_classifier.confusion_matrix.plot()

